Calls the FootyStats API and returns the league data of selected leagues for selected season (default current season)  
Stores all leagues to a single raw output csv 'all_league_matches_data_raw' for next stage processing

- next stage: extract 'incomplete', filter cols (odds_ft_under2.5, odds_ft_over2.5, odds_ft_1, odds_ft_x, odds_ft_2, date_unix, home_name,away_name,league_name)  

In [1]:
import streamlit as st
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
load_dotenv()

LEAGUE_IDS = {
   'eng1': 12325,
   'spain1': 12316,
   'italy1': 12530,
   'fra1': 12337,
   'ger1': 12529,
#   'swe1': 10969,
#   'nor1': 10976,
   'sa1': 13284
}

# Get API key from Streamlit secrets
API_KEY = os.getenv('API_KEY_FOOTYSTATS')

API call to return as individual dfs for each league

In [3]:
# def get_match_data(league_id, api_key, page_no):
#     base_url = "https://api.football-data-api.com/league-matches"
    
#     params = {
#         'page': page_no,
#         'key': api_key,
#         'league_id': league_id
#     }
    
#     response = requests.get(base_url, params=params)
    
#     if response.status_code == 200:
#         data = response.json()
#         return data.get('data', [])  # Extract 'data' key from the response
#     else:
#         print(f"Error: {response.status_code}")
#         return []


# page_no = 1

# all_dataframes = {}

# for league_name, league_id in LEAGUE_IDS.items():
#     page_no = 1
#     league_data = []

#     # max 200 rows will be returned per page so loop through each page whilst page contains more rows
#     while True:
#         data = get_match_data(league_id, api_key, page_no)
        
#         if data:
#             league_data.extend(data)
#             page_no += 1
#             time.sleep(5)
#         else:
#             break

#     if league_data:
#         # Convert extracted match data to Pandas DataFrame
#         df = pd.DataFrame(league_data)
#         all_dataframes[league_name] = df
#     else:
#         print("Failed to fetch match data.")

# if all_dataframes:
#     # Save each DataFrame to a separate file or perform further processing as needed
#     for league_name, df in all_dataframes.items():
#         df.to_csv(f'#####', index=False)
# else:
#     print("Failed to fetch match data for any league.")

Store output as a single (combined) df

In [4]:
def get_match_data(league_id, API_KEY, page_no):
    base_url = "https://api.football-data-api.com/league-matches"
    
    params = {
        'page': page_no,
        'key': API_KEY,
        'league_id': league_id
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('data', [])  # Extract 'data' key from the response
    else:
        print(f"Error: {response.status_code}")
        return []


# Initialize an empty list to store data from all leagues
all_league_data = []

for league_name, league_id in LEAGUE_IDS.items():
    page_no = 1
    league_data = []

    # Loop through each page while there is data
    while True:
        data = get_match_data(league_id, API_KEY, page_no)
        
        if data:
            league_data.extend(data)  # Append page data to league_data
            page_no += 1
            time.sleep(5)  # Delay to avoid API rate limits
        else:
            break

    if league_data:
        # Convert match data for the current league to a DataFrame
        df = pd.DataFrame(league_data)
        df['league_name'] = league_name  # Add a column for the league name
        all_league_data.append(df)  # Add the league DataFrame to the list
    else:
        print(f"Failed to fetch match data for {league_name}.")


Error: 500
Error: 500
Error: 500
Error: 500
Error: 500
Error: 500


In [5]:
# Concatenate all data into a single DataFrame
if all_league_data:
    combined_df = pd.concat(all_league_data, ignore_index=True)

    # Define the directory and file path to save the CSV
    save_dir = r'C:\Users\MikeD\Streamlit_BK_App\data\raw'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    file_path = os.path.join(save_dir, 'all_leagues_match_data_raw.csv')

    # Save the concatenated DataFrame to CSV
    combined_df.to_csv(file_path, index=False)
    print(f"Data successfully saved to {file_path}")
else:
    print("Failed to fetch match data for any league.")

Data successfully saved to C:\Users\MikeD\Streamlit_BK_App\data\raw\all_leagues_match_data_raw.csv


In [6]:
combined_df.tail()

,id,homeID,awayID,season,status,roundID,game_week,revised_game_week,homeGoals,awayGoals,homeGoalCount,awayGoalCount,totalGoalCount,team_a_corners,team_b_corners,totalCornerCount,team_a_offsides,team_b_offsides,team_a_yellow_cards,team_b_yellow_cards,team_a_red_cards,team_b_red_cards,team_a_shotsOnTarget,team_b_shotsOnTarget,team_a_shotsOffTarget,team_b_shotsOffTarget,team_a_shots,team_b_shots,team_a_fouls,team_b_fouls,team_a_possession,team_b_possession,refereeID,coach_a_ID,coach_b_ID,stadium_name,stadium_location,team_a_cards_num,team_b_cards_num,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over05,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_ft_under05,odds_ft_under15,odds_ft_under25,odds_ft_under35,odds_ft_under45,odds_btts_yes,odds_btts_no,odds_team_a_cs_yes,odds_team_a_cs_no,odds_team_b_cs_yes,odds_team_b_cs_no,odds_doublechance_1x,odds_doublechance_12,odds_doublechance_x2,odds_1st_half_result_1,odds_1st_half_result_x,odds_1st_half_result_2,odds_2nd_half_result_1,odds_2nd_half_result_x,odds_2nd_half_result_2,odds_dnb_1,odds_dnb_2,odds_corners_over_75,odds_corners_over_85,odds_corners_over_95,odds_corners_over_105,odds_corners_over_115,odds_corners_under_75,odds_corners_under_85,odds_corners_under_95,odds_corners_under_105,odds_corners_under_115,odds_corners_1,odds_corners_x,odds_corners_2,odds_team_to_score_first_1,odds_team_to_score_first_x,odds_team_to_score_first_2,odds_win_to_nil_1,odds_win_to_nil_2,odds_1st_half_over05,odds_1st_half_over15,odds_1st_half_over25,odds_1st_half_over35,odds_1st_half_under05,odds_1st_half_under15,odds_1st_half_under25,odds_1st_half_under35,odds_2nd_half_over05,odds_2nd_half_over15,odds_2nd_half_over25,odds_2nd_half_over35,odds_2nd_half_under05,odds_2nd_half_under15,odds_2nd_half_under25,odds_2nd_half_under35,odds_btts_1st_half_yes,odds_btts_1st_half_no,odds_btts_2nd_half_yes,odds_btts_2nd_half_no,overallGoalCount,ht_goals_team_a,ht_goals_team_b,goals_2hg_team_a,goals_2hg_team_b,GoalCount_2hg,HTGoalCount,date_unix,winningTeam,no_home_away,btts_potential,btts_fhg_potential,btts_2hg_potential,goalTimingDisabled,attendance,corner_timings_recorded,card_timings_recorded,team_a_fh_corners,team_b_fh_corners,team_a_2h_corners,team_b_2h_corners,corner_fh_count,corner_2h_count,team_a_fh_cards,team_b_fh_cards,team_a_2h_cards,team_b_2h_cards,total_fh_cards,total_2h_cards,attacks_recorded,team_a_dangerous_attacks,team_b_dangerous_attacks,team_a_attacks,team_b_attacks,team_a_xg,team_b_xg,total_xg,team_a_penalties_won,team_b_penalties_won,team_a_penalty_goals,team_b_penalty_goals,team_a_penalty_missed,team_b_penalty_missed,pens_recorded,goal_timings_recorded,team_a_0_10_min_goals,team_b_0_10_min_goals,team_a_corners_0_10_min,team_b_corners_0_10_min,team_a_cards_0_10_min,team_b_cards_0_10_min,throwins_recorded,team_a_throwins,team_b_throwins,freekicks_recorded,team_a_freekicks,team_b_freekicks,goalkicks_recorded,team_a_goalkicks,team_b_goalkicks,o45_potential,o35_potential,o25_potential,o15_potential,o05_potential,o15HT_potential,o05HT_potential,o05_2H_potential,o15_2H_potential,corners_potential,offsides_potential,cards_potential,avg_potential,home_url,home_image,home_name,away_url,away_image,away_name,home_ppg,away_ppg,pre_match_home_ppg,pre_match_away_ppg,pre_match_teamA_overall_ppg,pre_match_teamB_overall_ppg,u45_potential,u35_potential,u25_potential,u15_potential,u05_potential,corners_o85_potential,corners_o95_potential,corners_o105_potential,team_a_xg_prematch,team_b_xg_prematch,total_xg_prematch,match_url,competition_id,matches_completed_minimum,over05,over15,over25,over35,over45,over55,btts,homeGoals_timings,awayGoals_timings,league_name
1987,7638623,1264,688962,2024/2025,incomplete,112779,0,-1,[],[],0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,-1.0,-1.0,Mpumalanga Stadium (Mpumalanga),,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0